In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [11]:
class MyModel(nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
torch_model = MyModel()
torch_input = torch.randn(1, 1, 32, 32)

In [27]:
onnx_program = torch.onnx.dynamo_export(torch_model, torch_input)

c:\Users\thn12\miniconda3\envs\train_export\lib\site-packages\torch\onnx\_internal\_exporter_legacy.py:101: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


OnnxExporterError: Failed to export the model to ONNX. Generating SARIF report at 'report_dynamo_export.sarif'. SARIF is a standard format for the output of static analysis tools. SARIF logs can be loaded in VS Code SARIF viewer extension, or SARIF web viewer (https://microsoft.github.io/sarif-web-component/). Please report a bug on PyTorch Github: https://github.com/pytorch/pytorch/issues

In [ ]:

torch.onnx.export(
    torch_model,               
    torch_input,               
    "my_image_classifier.onnx",
    export_params=True,        
    opset_version=10,          
    do_constant_folding=True,  
    input_names=['input'],     
    output_names=['output'],
    dynamic_axes={'input' : {0 : 'batch_size'}, 
                'output' : {0 : 'batch_size'}}    
)

In [23]:
import onnx
onnx_model = onnx.load("my_image_classifier.onnx")
onnx.checker.check_model(onnx_model)

In [25]:
import onnxruntime

onnx_input = [torch_input]
print(f"Input length: {len(onnx_input)}")
print(f"Sample input: {onnx_input}")

ort_session = onnxruntime.InferenceSession("./my_image_classifier.onnx", providers=['CPUExecutionProvider'])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}

# onnxruntime returns a list of outputs
onnxruntime_outputs = ort_session.run(None, onnxruntime_input)[0]

Input length: 1
Sample input: [tensor([[[[ 0.6942,  0.1307,  0.4381,  ...,  1.1517, -2.0320,  0.8644],
          [-0.9993, -0.3215,  0.1849,  ...,  0.5924,  0.6825, -0.7125],
          [ 1.1596,  0.4333, -1.0273,  ..., -0.2625,  1.4281, -1.3317],
          ...,
          [ 0.3655, -0.1390,  0.9420,  ..., -0.8655,  1.5685, -1.9397],
          [-0.7453, -0.5754, -0.8150,  ..., -1.4323,  0.6022, -0.3348],
          [-0.1405,  0.5347, -0.9429,  ...,  0.7457, -0.5872,  0.2565]]]])]


In [26]:
torch_outputs = torch_model(torch_input)

assert len(torch_outputs) == len(onnxruntime_outputs)
for torch_output, onnxruntime_output in zip(torch_outputs, onnxruntime_outputs):
    torch.testing.assert_close(torch_output, torch.tensor(onnxruntime_output))

print("PyTorch and ONNX Runtime output matched!")
print(f"Output length: {len(onnxruntime_outputs)}")
print(f"Sample output: {onnxruntime_outputs}")

PyTorch and ONNX Runtime output matched!
Output length: 1
Sample output: [[ 0.09706248 -0.02956782 -0.10485043 -0.00360697 -0.02275606  0.11563597
  -0.02164603 -0.15843765  0.12351643  0.15656006]]
